In [24]:
from math import log, cos, sin, pi
N = 8
def embedding(coeffs):
    """Computes a variant of the canonical embedding on the given coefficients.

    Computes the canonical embedding which consists of evaluating a given polynomial at roots of unity
    that are indexed 1 (mod 4), w, w^5, w^9, ...
    The evaluations are returned in the order: w, w^5, w^(5^2), ...

    Args:
        coeffs (list): List of complex numbers to transform.

    Returns:
        List of transformed coefficients.
    """
    fft_length = N
    num_slots = fft_length // 4
    roots_of_unity = [0] * fft_length
    roots_of_unity_inv = [0] * fft_length
    for i in range(fft_length):
        angle = 2 * pi * i / fft_length
        roots_of_unity[i] = complex(cos(angle), sin(angle))
        roots_of_unity_inv[i] = complex(cos(-angle), sin(-angle))
    rot_group = [1] * num_slots
    for i in range(1, num_slots):
        rot_group[i] = (5 * rot_group[i - 1]) % fft_length
    print(rot_group)
    num_coeffs = len(coeffs)
    result = coeffs
    log_num_coeffs = int(log(num_coeffs, 2))
    for logm in range(1, log_num_coeffs + 1):
        idx_mod = 1 << (logm + 2)
        gap = fft_length // idx_mod
        for j in range(0, num_coeffs, (1 << logm)):
            for i in range(1 << (logm - 1)):
                index_even = j + i
                index_odd = j + i + (1 << (logm - 1))

                rou_idx = (rot_group[i] % idx_mod) * gap
                omega_factor = roots_of_unity[rou_idx] * result[index_odd]

                butterfly_plus = result[index_even] + omega_factor
                butterfly_minus = result[index_even] - omega_factor

                result[index_even] = butterfly_plus
                result[index_odd] = butterfly_minus
    return result
embedding([1,2,3,4])

[1, 5]


[(8.242640687119284+4.242640687119286j),
 (-0.24264068711928544-4.242640687119286j),
 (-3.414213562373095-1.4142135623730951j),
 (-0.5857864376269049+1.4142135623730951j)]

In [23]:
from math import log, cos, sin, pi

# Define the FFT length as a constant (8 in this case)
N = 8

def embedding2(coeffs):
    """Computes a variant of the canonical embedding on the given coefficients.

    Computes the canonical embedding which consists of evaluating a given polynomial at roots of unity
    that are indexed 1 (mod 4), w, w^5, w^9, ...
    The evaluations are returned in the order: w, w^5, w^(5^2), ...

    Args:
        coeffs (list): List of complex numbers to transform.

    Returns:
        List of transformed coefficients.
    """

    # FFT parameters
    fft_length = N  # Length of the FFT
    num_slots = fft_length // 4  # Number of rotation slots (used in cyclic group)

    # Arrays for roots of unity (and inverse roots for inverse FFT if needed)
    roots_of_unity = [0] * fft_length  # Forward roots of unity
    roots_of_unity_inv = [0] * fft_length  # Inverse roots of unity

    # Compute the roots of unity for FFT (points on the unit circle in the complex plane)
    for i in range(fft_length):
        angle = 2 * pi * i / fft_length  # Angle for each root of unity
        roots_of_unity[i] = complex(cos(angle), sin(angle))  # Compute root of unity at index i
        roots_of_unity_inv[i] = complex(cos(-angle), sin(-angle))  # Compute inverse root of unity

    # Compute the rotation group used in FFT
    rot_group = [1] * num_slots  # Initialize rotation group array
    for i in range(1, num_slots):
        rot_group[i] = (5 * rot_group[i - 1]) % fft_length  # Compute each element using modular arithmetic
    print(rot_group)  # Print the rotation group (useful for debugging)

    # Initialize parameters for the FFT loop
    num_coeffs = len(coeffs)  # Number of coefficients (input length)
    result = coeffs  # Initialize result with the input coefficients
    log_num_coeffs = int(log(num_coeffs, 2))  # Logarithm base 2 of the number of coefficients

    # Perform the FFT computation using a log(N) loop
    for logm in range(1, log_num_coeffs + 1):
        idx_mod = 1 << (logm + 2)  # Calculate modulus for rotation group indices
        gap = fft_length // idx_mod  # Determine the gap between elements in the roots of unity

        # Inner loop to perform butterfly operations for FFT
        for j in range(0, num_coeffs, (1 << logm)):
            for i in range(1 << (logm - 1)):
                # Indices for even and odd parts of the butterfly computation
                index_even = j + i
                index_odd = j + i + (1 << (logm - 1))

                # Calculate the twiddle factor (omega factor) from the roots of unity
                rou_idx = (rot_group[i] % idx_mod) * gap
                omega_factor = roots_of_unity[rou_idx] * result[index_odd]

                # Compute the butterfly operation (combining even and odd parts)
                butterfly_plus = result[index_even] + omega_factor  # Addition part
                butterfly_minus = result[index_even] - omega_factor  # Subtraction part

                # Update the result array with the butterfly results
                result[index_even] = butterfly_plus
                result[index_odd] = butterfly_minus

    # Return the transformed coefficients (result after applying FFT)
    return result
embedding2([1,2,3,4])

[1, 5]


[(8.242640687119284+4.242640687119286j),
 (-0.24264068711928544-4.242640687119286j),
 (-3.414213562373095-1.4142135623730951j),
 (-0.5857864376269049+1.4142135623730951j)]